In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
LOCAL_FAST_AI = '/Users/krishnakalyan3/Educational/FastAI/fastai/'
AWS_FAST_AI = '/home/ubuntu/fastai'
sys.path.append(AWS_FAST_AI)

In [3]:
from fastai.imports import *
from glob import iglob
import pprint 
from fastai.transforms import * 
from fastai.conv_learner import * 
from fastai.model import *
from fastai.dataset import * 
from fastai.sgdr import *
from fastai.plots import *
from fastai.models.nasnet import nasnetalarge

PATH = '/home/ubuntu/Plant/data/'

Create Label File

Train Model

In [4]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)
n

5544

In [5]:
f_model = resnext50
sz=250
tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)

In [32]:
data = ImageClassifierData.from_csv(PATH, folder='train', csv_fname=f'{PATH}labels.csv',
                                    tfms=tfms, val_idxs=[0], test_name='test', bs=12)

In [10]:
data = ImageClassifierData.from_csv(PATH, folder='train', csv_fname=f'{PATH}labels.csv',
                                    tfms=tfms, val_idxs=[0], test_name='test', bs=12)
learn = ConvLearner.pretrained(f_model, data, precompute=True)

100%|██████████| 1/1 [00:00<00:00, 36.22it/s]


In [11]:
learn.fit(1e-2, 3)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       0.92826  0.11061  1.     ]                         
[ 1.       0.79673  0.05552  1.     ]                         
[ 2.       0.62162  0.03857  1.     ]                         



In [12]:
lr = 1e-2
learn.unfreeze()
lr=np.array([lr/18,lr/6,lr/2])
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.60765  0.05381  1.     ]                        
[ 1.       0.61196  0.01889  1.     ]                        
[ 2.       0.53334  0.03731  1.     ]                        
[ 3.       0.51664  0.03415  1.     ]                        
[ 4.       0.51728  0.03149  1.     ]                        
[ 5.       0.44519  0.04001  1.     ]                        
[ 6.       0.44658  0.02674  1.     ]                        



In [13]:
learn.fit(lr, 3, cycle_len=2, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=14), HTML(value='')))

[ 0.       0.50917  0.05668  1.     ]                        
[ 1.       0.43548  0.03929  1.     ]                        
[ 2.       0.48881  0.01871  1.     ]                        
[ 3.       0.43693  0.01219  1.     ]                        
[ 4.       0.40739  0.02105  1.     ]                        
[ 5.       0.42221  0.02311  1.     ]                        
[ 6.       0.40867  0.03343  1.     ]                        
[ 7.       0.43012  0.02512  1.     ]                        
[ 8.       0.44835  0.00757  1.     ]                        
[ 9.       0.41278  0.01872  1.     ]                        
[ 10.        0.44042   0.00679   1.     ]                    
[ 11.        0.40451   0.01447   1.     ]                    
[ 12.        0.39247   0.01703   1.     ]                    
[ 13.        0.30707   0.01041   1.     ]                    



In [14]:
learn.fit(lr, 3, cycle_len=2, cycle_mult=4)

HBox(children=(IntProgress(value=0, description='Epoch', max=42), HTML(value='')))

[ 0.       0.44754  0.0177   1.     ]                        
[ 1.       0.40383  0.01559  1.     ]                        
[ 2.       0.43397  0.0174   1.     ]                        
[ 3.       0.42141  0.06177  1.     ]                        
[ 4.       0.36517  0.01107  1.     ]                        
[ 5.       0.35082  0.0173   1.     ]                        
[ 6.       0.3557   0.02256  1.     ]                        
[ 7.       0.35511  0.01677  1.     ]                        
[ 8.       0.34663  0.01712  1.     ]                        
[ 9.       0.33904  0.01575  1.     ]                        
[ 10.        0.37884   0.01457   1.     ]                    
[ 11.        0.42951   0.00592   1.     ]                    
[ 12.        0.39617   0.00526   1.     ]                    
[ 13.        0.37891   0.01809   1.     ]                    
[ 14.        0.31894   0.00568   1.     ]                    
[ 15.        0.32477   0.01919   1.     ]                    
[ 16.   

In [15]:
learn.save(f'resnext_50_all_data')

Metrics

In [20]:
log_preds_test = learn.predict(is_test=True)
log_preds_test.shape

(793, 12)

In [21]:
yhat_test = np.argmax(log_preds_test, axis=1)
lp_prob_test = np.exp(yhat_test)

In [22]:
preds_classes = [data.classes[i].replace("_", " ") for i in yhat_test]
submission = pd.DataFrame({'file': os.listdir(f'{PATH}test'), 'species': preds_classes})

ValueError: arrays must all be same length

In [49]:
#submission.to_csv(PATH + 'submission_10.csv', index=False)

In [23]:
log_preds_test = learn.predict(is_test=True)
log_preds_test = np.argmax(log_preds_test, axis=1)

In [24]:
probs_1 = np.exp(log_preds_test)

In [25]:
preds_classes = [data.classes[i].replace("_", " ") for i in log_preds_test]

In [33]:
len(preds_classes)

793

In [34]:
len(os.listdir(f'{PATH}test'))

794

In [26]:
submission = pd.DataFrame({'file': os.listdir(f'{PATH}test'), 'species': preds_classes})
submission.to_csv(PATH + 'submission_2.csv', index=False)

ValueError: arrays must all be same length